# Implementation of the Kuribayashi BERT minus model

## libraries

In [1]:
!pip install transformers --upgrade
!pip install ipywidgets
!pip install IProgress
!pip install datasets
!pip install torch-lr-finder

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import transformers
from transformers import BertTokenizer, BertConfig
from transformers import BertModel, BertForSequenceClassification
from transformers import BatchEncoding, default_data_collator, DataCollatorWithPadding

import torch
import torch.nn as nn

import numpy as np

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

import datasets

from torch.utils.data import DataLoader

from tqdm import tqdm
from operator import itemgetter

In [3]:
print(transformers.__version__)

4.26.0


## tokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [5]:
# tokenizer.model_max_length = 256

## data

In [6]:
# DATA_FOLDER = '/notebooks/Data/bert_sequence_classification'
DATA_FILE = '/notebooks/KURI-BERT/notebooks/full_formula_w_fts/Stance_Recognition_Task/pe_dataset_for_bert_minus_w_fts_for_stance_rec.pt'
RESULTS_FOLDER = '/notebooks/KURI-BERT/results'

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
device

device(type='cuda')

## Load data

In [9]:
dataset = torch.load(DATA_FILE)

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['paragraph', 'paragraph_components_list_x', 'paragraph_labels_list_x', 'paragraph_markers_list_x', 'split_x', 'essay_nr_x', 'structural_fts_as_text_combined_x', 'paragraph_labels_numeric_x', 'paragraph_components_list_y', 'paragraph_labels_list_y', 'paragraph_markers_list_y', 'split_y', 'essay_nr_y', 'structural_fts_as_text_combined_y', 'paragraph_labels_numeric_y', 'full_paragraph', 'ac_spans', 'fts_spans', 'am_spans'],
        num_rows: 859
    })
    test: Dataset({
        features: ['paragraph', 'paragraph_components_list_x', 'paragraph_labels_list_x', 'paragraph_markers_list_x', 'split_x', 'essay_nr_x', 'structural_fts_as_text_combined_x', 'paragraph_labels_numeric_x', 'paragraph_components_list_y', 'paragraph_labels_list_y', 'paragraph_markers_list_y', 'split_y', 'essay_nr_y', 'structural_fts_as_text_combined_y', 'paragraph_labels_numeric_y', 'full_paragraph', 'ac_spans', 'fts_spans', 'am_spans'],
        num_rows: 274
    })

In [11]:
dataset['train']['paragraph_labels_numeric_x'][0]

[1]

### preprocessing

In [12]:
MAX_LENGTH = 0
MAX_SPAN = 0

for split in ['train', 'test', 'validation']:
    
    for col_name in ['fts_spans']:
        
        for x in dataset[split][col_name]:
            
            # print(x)
            
#             try:
#                 max(x,key=itemgetter(1))[1] > MAX_SPAN
#                 MAX_SPAN = max(x,key=itemgetter(1))[1]
#                 MAX_SPAN = min(MAX_SPAN, tokenizer.model_max_length - 2)
                
#             except:
                
#                 pass
            
            if max(x,key=itemgetter(1))[1] > MAX_SPAN:
                
                MAX_SPAN = max(x,key=itemgetter(1))[1]
                MAX_SPAN = min(MAX_SPAN, tokenizer.model_max_length - 2)
            
            if len(x) > MAX_LENGTH:
                
                MAX_LENGTH = len(x)

In [13]:
MAX_SPAN

312

In [14]:
tokenizer.model_max_length

512

In [15]:
def get_padding(batch, padding_target):    
    
    if padding_target == 'am_spans':
        
        col_name = 'am_spans'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'ac_spans':
        
        col_name = 'ac_spans'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'fts_spans':
        
        col_name = 'fts_spans'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'label':
    
        col_name = 'paragraph_labels_numeric_x'
        padding_val = [-100] # -1 previously       
        max_length = MAX_LENGTH # max([len(l) for l in batch[col_name]]) # cause some batch had 4 x 10

    padded_spans = []

    for idx, span in enumerate(batch[col_name]):

        padded_span = batch[col_name][idx] + (max_length - len(span)) * padding_val
        padded_spans.append(padded_span)

    return padded_spans         

In [16]:
def get_combined_spans(am_spans_ll, ac_spans_ll, fts_spans_ll):
    
    spans_ll = []
    
    for am_spans, ac_spans, fts_spans in zip(am_spans_ll, ac_spans_ll, fts_spans_ll):
        
        spans = []
        
        for am_span, ac_span, fts_span in zip(am_spans, ac_spans, fts_spans):
            
            for idx in [0,1]:
                
                if am_span[idx] > MAX_SPAN:
                    am_span[idx] = MAX_SPAN
                if ac_span[idx] > MAX_SPAN:
                    ac_span[idx] = MAX_SPAN
                if fts_span[idx] > MAX_SPAN:
                    fts_span[idx] = MAX_SPAN

            span = [am_span, ac_span, fts_span]
            spans.extend(span)
            
        spans_ll.append(spans)

    return spans_ll

### tokenize 

In [17]:
# max_length = 200 for use in the max_length in the tokenizer so that the things are of equal dim.

In [18]:
def tokenize(batch):
    
    tokenized_text = tokenizer(batch['full_paragraph'], truncation=True, padding=True, max_length=512)
    tokenized_text['label'] = get_padding(batch, 'label')
    tokenized_text['am_spans'] = get_padding(batch, 'am_spans')
    tokenized_text['ac_spans'] = get_padding(batch, 'ac_spans')
    tokenized_text['fts_spans'] = get_padding(batch, 'fts_spans')
    tokenized_text['spans'] = get_combined_spans(tokenized_text['am_spans'], tokenized_text['ac_spans'], tokenized_text['fts_spans'])      
    
    return tokenized_text

In [19]:
dataset = dataset.map(tokenize, batched=True, batch_size=len(dataset['train']))

Parameter 'function'=<function tokenize at 0x7f20204609d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ac_spans', 'am_spans', 'attention_mask', 'essay_nr_x', 'essay_nr_y', 'fts_spans', 'full_paragraph', 'input_ids', 'label', 'paragraph', 'paragraph_components_list_x', 'paragraph_components_list_y', 'paragraph_labels_list_x', 'paragraph_labels_list_y', 'paragraph_labels_numeric_x', 'paragraph_labels_numeric_y', 'paragraph_markers_list_x', 'paragraph_markers_list_y', 'spans', 'split_x', 'split_y', 'structural_fts_as_text_combined_x', 'structural_fts_as_text_combined_y', 'token_type_ids'],
        num_rows: 859
    })
    test: Dataset({
        features: ['ac_spans', 'am_spans', 'attention_mask', 'essay_nr_x', 'essay_nr_y', 'fts_spans', 'full_paragraph', 'input_ids', 'label', 'paragraph', 'paragraph_components_list_x', 'paragraph_components_list_y', 'paragraph_labels_list_x', 'paragraph_labels_list_y', 'paragraph_labels_numeric_x', 'paragraph_labels_numeric_y', 'paragraph_markers_list_x', 'paragraph_markers_list_y', 'spans', 'split_x'

In [21]:
dataset['train'][0]['spans']

[[0, 1],
 [4, 8],
 [11, 19],
 [-1, -1],
 [21, 28],
 [31, 39],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1]]

In [22]:
len(dataset['train'][0]['spans'])

36

In [23]:
dataset['train'][1]['label']

[0, 1, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100]

In [24]:
dataset['train'].features

{'ac_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'am_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'essay_nr_x': Value(dtype='string', id=None),
 'essay_nr_y': Value(dtype='string', id=None),
 'fts_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'full_paragraph': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_components_list_x': Sequence(feature=Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), length=-1, id=None),
 'paragraph_components_list_y': Sequence(feature=Sequence(f

In [25]:
dataset['test'].features

{'ac_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'am_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'essay_nr_x': Value(dtype='string', id=None),
 'essay_nr_y': Value(dtype='string', id=None),
 'fts_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'full_paragraph': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_components_list_x': Sequence(feature=Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), length=-1, id=None),
 'paragraph_components_list_y': Sequence(feature=Sequence(f

In [26]:
dataset['test'].features['spans'] = datasets.Array2D(shape=(36, 2), dtype="int32")
dataset['train'].features['spans'] = datasets.Array2D(shape=(36, 2), dtype="int32")
dataset['validation'].features['spans'] = datasets.Array2D(shape=(36, 2), dtype="int32")

In [27]:
dataset = dataset.map(lambda batch: batch, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [28]:
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'spans', 'label'])

## span representation function

In [29]:
def minus_one(t):
    
    return torch.where(t == 0, 0, t-1)

def plus_one(t):
    
    return torch.where(t >= MAX_SPAN, MAX_SPAN, t+1) # changed from t == MAX_SPAN

In [30]:
def get_span_representations(outputs, spans):
    
    #print('spans shape', spans.shape)
    
    batch_size = spans.shape[0]
    
    nr_span_indices = spans.shape[1]
    
    #print('nr span indices', nr_span_indices)
    
    # nr_span_indices = 24 # xxx. hardcode just to check
    
    idx_l_ams = range(0, nr_span_indices, 3) # [0,2,4,6 etc]
    idx_l_acs = range(1, nr_span_indices, 3) # [1,3,5,7 etc]
    idx_l_fts = range(2, nr_span_indices, 3)
    
    am_spans = spans[:, idx_l_ams, :] + 1 # adds 1 to all span indices (both am and ac) to offset for the CLS token in the input_ids.
    ac_spans = spans[:, idx_l_acs, :] + 1
    fts_spans = spans[:, idx_l_fts, :] + 1
    
    
    # fix for [-1, +1] problem
    
    am_spans_minus_one = minus_one(am_spans) # xxx. added to solve bug 2
    am_spans_plus_one = plus_one(am_spans) # xxx. added to solve bug 2
    
    ac_spans_minus_one = minus_one(ac_spans) # xxx. added to solve bug 2
    ac_spans_plus_one = plus_one(ac_spans) # xxx. added to solve bug 2
    
    fts_spans_minus_one = minus_one(fts_spans) # xxx. added to solve bug 2
    fts_spans_plus_one = plus_one(fts_spans) # xxx. added to solve bug 2
    
    
    am_spans = am_spans.flatten(start_dim=1)
    ac_spans = ac_spans.flatten(start_dim=1)
    fts_spans = fts_spans.flatten(start_dim=1)
    
    nr_adus = ac_spans.shape[1] // 2
    
    
    am_spans_minus_one = am_spans_minus_one.flatten(start_dim=1) # xxx. added to solve bug 2
    am_spans_plus_one = am_spans_plus_one.flatten(start_dim=1) # xxx. added to solve bug 2
    
    ac_spans_minus_one = ac_spans_minus_one.flatten(start_dim=1) # xxx. added to solve bug 2
    ac_spans_plus_one = ac_spans_plus_one.flatten(start_dim=1) # xxx. added to solve bug 2
    
    fts_spans_minus_one = fts_spans_minus_one.flatten(start_dim=1) # xxx. added to solve bug 2
    fts_spans_plus_one = fts_spans_plus_one.flatten(start_dim=1) # xxx. added to solve bug 2
    
    ############# FOR AMs #################
    
    outputs_am = outputs[:,am_spans,:]
    #print('outputs am juste directly from outputs:', outputs_am.shape)
    outputs_am = torch.cat([outputs_am[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_am = outputs_am.reshape(batch_size, nr_adus * 2, -1)
    
    #print('outputs_ am after reshape:', outputs_am.shape)
    
    
    
    outputs_am_minus_one = outputs[:,am_spans_minus_one,:] # xxx. added to solve bug 2
    outputs_am_minus_one = torch.cat([outputs_am_minus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_am_minus_one = outputs_am_minus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    outputs_am_plus_one = outputs[:,am_spans_plus_one,:] # xxx. added to solve bug 2
    outputs_am_plus_one = torch.cat([outputs_am_plus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_am_plus_one = outputs_am_plus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    
    
    ### Now that we have outputs_am i.e. outputs at am_span indices, now create the four Kuri forumlas for AMs
    
    # ============== the corrected 1st one =================== 
    
    outputs_am_first_term = torch.cat([outputs_am[:,i+1,:] - outputs_am_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1) # i + 1 here means j in kuri, i here means i in kuri
    outputs_am_first_term = outputs_am_first_term.reshape(batch_size, -1, 768)
    
     # ============== the corrected 2nd one ==================
    
    outputs_am_second_term = torch.cat([outputs_am[:,i,:] - outputs_am_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run # changed from +1 to +2 to ensure +2 is not a problem for AMs
    outputs_am_second_term = outputs_am_second_term.reshape(batch_size, -1, 768)
    
    
        # ============== the corrected third one ================== 
    
    outputs_am_third_term = torch.cat([outputs_am_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_am_third_term = outputs_am_third_term.reshape(batch_size, -1, 768)
    

        # ============== the corrected fourth one ==================
    
    outputs_am_fourth_term = torch.cat([outputs_am_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run # changed from +1 to +2 to ensure +2 is not a problem for AMs
    outputs_am_fourth_term = outputs_am_fourth_term.reshape(batch_size, -1, 768)
    
    # ============== NOW CONCATENATE THEM =========
    
    
    am_minus_representations = torch.cat([outputs_am_first_term, outputs_am_second_term, outputs_am_third_term, outputs_am_fourth_term], dim=-1)   
    
    
    
    ### am minus span representation according to kuribayashi paper is now above.
    
    
    outputs_ac = outputs[:,ac_spans,:]
    outputs_ac = torch.cat([outputs_ac[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_ac = outputs_ac.reshape(batch_size, nr_adus * 2, -1)
    
    
    outputs_ac_minus_one = outputs[:,ac_spans_minus_one,:] # xxx. added to solve bug 2
    outputs_ac_minus_one = torch.cat([outputs_ac_minus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_ac_minus_one = outputs_ac_minus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    outputs_ac_plus_one = outputs[:,ac_spans_plus_one,:] # xxx. added to solve bug 2
    outputs_ac_plus_one = torch.cat([outputs_ac_plus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_ac_plus_one = outputs_ac_plus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    ### Now that we have outputs_ac i.e. outputs at ac_span indices, now create the four Kuri forumlas for ACs
    
    
    # ============== the corrected first one ===================
    
    outputs_ac_first_term = torch.cat([outputs_ac[:,i+1,:] - outputs_ac_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_ac_first_term = outputs_ac_first_term.reshape(batch_size, -1, 768)
    
        # ============== the correct second one ==================
    
    outputs_ac_second_term = torch.cat([outputs_ac[:,i,:] - outputs_ac_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run
    outputs_ac_second_term = outputs_ac_second_term.reshape(batch_size, -1, 768)
    
    
    # ============== the corrected third term ==================
    
    outputs_ac_third_term = torch.cat([outputs_ac_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_ac_third_term = outputs_ac_third_term.reshape(batch_size, -1, 768)

    
     # ============== the corrected fourth term ================== xxx. for .788
    
    outputs_ac_fourth_term = torch.cat([outputs_ac_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run
    outputs_ac_fourth_term = outputs_ac_fourth_term.reshape(batch_size, -1, 768)
    
    # ============== NOW CONCATENATE THEM =========
    
    
    ac_minus_representations = torch.cat([outputs_ac_first_term, outputs_ac_second_term, outputs_ac_third_term, outputs_ac_fourth_term], dim=-1)   
    
    
    
    ### ac minus span representation according to kuribayashi paper is now above.
    
    
     ############# FOR Fts #################
    
    outputs_fts = outputs[:,fts_spans,:] # am spans for checking
    #print('outputs fts juste directly from the outputs:', outputs_fts.shape)
    outputs_fts = torch.cat([outputs_fts[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_fts = outputs_fts.reshape(batch_size, nr_adus * 2, -1)
    
    #print('output fts after reshape:', outputs_fts.shape)
    
    
    
    outputs_fts_minus_one = outputs[:,fts_spans_minus_one,:] # xxx. added to solve bug 2
    outputs_fts_minus_one = torch.cat([outputs_fts_minus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_fts_minus_one = outputs_fts_minus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    outputs_fts_plus_one = outputs[:,fts_spans_plus_one,:] # xxx. added to solve bug 2
    outputs_fts_plus_one = torch.cat([outputs_fts_plus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_fts_plus_one = outputs_fts_plus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    #print('outputs_fts_plus_one', outputs_fts_plus_one.shape)
    
    
    
    ### Now that we have outputs_am i.e. outputs at am_span indices, now create the four Kuri forumlas for AMs
    
    # ============== (batch_size x nr_adus x 768) =================== 
    
    outputs_fts_first_term = torch.cat([outputs_fts[:,i+1,:] - outputs_fts_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1) # i + 1 here means j in kuri, i here means i in kuri
    outputs_fts_first_term = outputs_fts_first_term.reshape(batch_size, -1, 768)
    #print('outputs_fts_first_term:', outputs_fts_first_term.shape)
    
     # ============== the corrected 2nd one ==================
    
    outputs_fts_second_term = torch.cat([outputs_fts[:,i,:] - outputs_fts_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_fts_second_term = outputs_fts_second_term.reshape(batch_size, -1, 768)
    #print('outputs_fts_second_term:', outputs_fts_second_term.shape)
    
        # ============== the corrected third one ================== 
    
    outputs_fts_third_term = torch.cat([outputs_fts_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_fts_third_term = outputs_fts_third_term.reshape(batch_size, -1, 768)
    

        # ============== the corrected fourth one ==================
    
    outputs_fts_fourth_term = torch.cat([outputs_fts_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_fts_fourth_term = outputs_fts_fourth_term.reshape(batch_size, -1, 768)
    #print('outputs_fts_fourth_term:', outputs_fts_fourth_term.shape)
    
    # ============== NOW CONCATENATE THEM =========
    
    
    fts_minus_representations = torch.cat([outputs_fts_first_term, outputs_fts_second_term, outputs_fts_third_term, outputs_fts_fourth_term], dim=-1)   
    
    
    
    ### fts minus span representation according to kuribayashi paper is now above.
    
#     print('am rep final:', am_minus_representations.shape)
#     print('ac rep final:', ac_minus_representations.shape)
#     print('fts rep final:', fts_minus_representations.shape)
    

    
    return am_minus_representations, ac_minus_representations, fts_minus_representations                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

### span representation function old

## custom BERT model

In [31]:
class CustomBERTKuri(nn.Module):

    def __init__(self, first_model, model_am, model_ac, model_fts, nr_classes):
        
        super(CustomBERTKuri, self).__init__()
        
        self.first_model = first_model
        
        self.intermediate_linear_am = nn.Linear(3072, 768)
        self.intermediate_linear_ac = nn.Linear(3072, 768) 
        self.intermediate_linear_fts = nn.Linear(3072, 768)
        
        self.model_am = model_am
        self.model_ac = model_ac
        self.model_fts = model_fts
        
        self.nr_classes = nr_classes
                
        self.fc = nn.Linear(self.model_am.config.hidden_size + self.model_ac.config.hidden_size + self.model_fts.config.hidden_size, self.nr_classes)
        # self.fc = nn.Linear(self.model_am.config.hidden_size + self.model_ac.config.hidden_size, self.nr_classes) 

    def forward(self, inputs):
        
        batch_tokenized, batch_spans = inputs 
        outputs = self.first_model(batch_tokenized)[0]
        #outputs = self.first_model(batch_tokenized, output_hidden_states=True)[1][12] # ** removed to correct error.
        # spans = batch_spans # remove this spans thing cause we are now giving it just the spans themselves.
        am_minus_representations, ac_minus_representations, fts_minus_representations = get_span_representations(outputs, batch_spans)
#         print('passed and gotten reps for all three successfully!')
#         print('am rep final in model:', am_minus_representations.shape)
#         print('ac rep final in model:', ac_minus_representations.shape)
#         print('fts rep final in model:', fts_minus_representations.shape)
        #am_minus_representations, ac_minus_representations = get_span_representations(outputs, batch_spans)
        
        # am_minus_representations = 
        
        
        #print('am minus rep:', am_minus_representations.shape)
        #print('ac minus rep:', ac_minus_representations.shape)
        #print('linear layer shape:', self.intermediate_linear_am.in_features)

        am_minus_representations = self.intermediate_linear_am(am_minus_representations)
        #print('passed first linear')
        ac_minus_representations = self.intermediate_linear_ac(ac_minus_representations)
        #print('passed second linear')
        fts_minus_representations = self.intermediate_linear_fts(fts_minus_representations)
        #print('passed third linear')
        
        #print('am minus rep:', am_minus_representations.shape)
        #print('ac minus rep:', ac_minus_representations.shape)
        #print('fts minus rep:', fts_minus_representations.shape)

        output_model_am = self.model_am(inputs_embeds = am_minus_representations)[0]
        output_model_ac = self.model_ac(inputs_embeds = ac_minus_representations)[0]
        output_model_fts = self.model_fts(inputs_embeds = fts_minus_representations)[0]

        adu_representations = torch.cat([output_model_am, output_model_ac, output_model_fts], dim=-1)
        #adu_representations = torch.cat([output_model_am, output_model_ac], dim=-1)
        # print("adu rep:", adu_representations.shape)
        output = self.fc(adu_representations)
        # print("model class output avant reshape:", output.shape)
        # output = output.reshape(-1, self.nr_classes)
        # print("model class output apres:", output.shape)
        return output

## Run

In [32]:
NB_EPOCHS = 40
BATCH_SIZE = 24

In [33]:
first_model = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
#first_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
#first_model.load_state_dict(torch.load('/notebooks/KURI-BERT/notebooks/full_formula_w_fts/icann_finetuned_work/link_identification_finetuned_model_new.pth'))
#first_model.load_state_dict(torch.load('/notebooks/KURI-BERT/notebooks/full_formula_w_fts/icann_finetuned_work/stance_rec_finetuned_model_new_single.pth'))

In [34]:
model_am = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [35]:
model_ac = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [36]:
model_fts = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [37]:
# custom_model = CustomBERTKuri(first_model, model_am, model_ac, model_fts, 3)
custom_model = CustomBERTKuri(first_model, model_am, model_ac, model_fts, 2)

In [38]:
custom_model.to(device)

CustomBERTKuri(
  (first_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [39]:
loss = nn.CrossEntropyLoss(ignore_index=- 100)

In [40]:
optimizer = torch.optim.AdamW(custom_model.parameters(), lr=0.0035111917342151295)

In [41]:
# 1.8738174228603844e-05
# 0.008111308307896872
# best learning rate found by the whole leslie business
# new best LR found= 9.999999999999997e-06

In [42]:
NR_BATCHES = len(dataset['train']) / BATCH_SIZE
num_training_steps = NB_EPOCHS * NR_BATCHES
num_warmup_steps = int(0.2 * num_training_steps)

In [43]:
def lr_lambda(current_step: int):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(
            0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps))
        )

    # return LambdaLR(optimizer, lr_lambda, last_epoch)

In [44]:
# commented for LR Finder. remove it from optimizer.
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

### create dataloaders

In [45]:
train_dataloader = DataLoader(dataset['train'], batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset['validation'], batch_size=BATCH_SIZE, shuffle=True)

In [46]:
# xxx. delete datasets for memory work
# del dataset

In [47]:
def flatten_list(list_of_lists):
    return [x for sublist in list_of_lists for x in sublist]

In [48]:
def remove_dummy_labels(test_preds, test_labels):
    
    idxes = []
    test_labels_l = []
    for idx, val in enumerate(test_labels):
        if val != -100:
            idxes.append(idx)
            test_labels_l.append(val)
    
    test_preds_l = []
    for idx, val in enumerate(test_preds):
        for good_idx in idxes:
            if idx == good_idx:
                test_preds_l.append(val)
        
    return test_preds_l, test_labels_l

In [49]:
for i, b in enumerate(train_dataloader):
    if i == 40:
        break

In [50]:
b

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[  101,  1999,  2344,  ...,     0,     0,     0],
         [  101,  2000, 16519,  ...,     0,     0,     0],
         [  101, 15847,  1010,  ...,     0,     0,     0],
         ...,
         [  101,  1999,  7680,  ...,     0,     0,     0],
         [  101,  2000,  4088,  ...,     0,     0,     0],
         [  101,  1999,  2804,  ...,     0,     0,     0]]),
 'label': tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0, -100, -100, -100],
         [   0,    0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100],
         [   0,    0,    0,    0, -100, -100, -100, -100, -100, -100, -100, -100],
         [   0,    0,    0,    0,    0, -100, -100, -100, -100, -100, -100, -100],
         [   0,    0,    0,    0

### training loop

In [51]:
from transformers import Trainer, TrainingArguments
from datasets import load_metric

In [52]:
import random
from torch_lr_finder import LRFinder

### LR Finder Leslie Smith 

## training 

In [53]:
def train(model, loss=None, optimizer=None, train_dataloader=None, val_dataloader=None, nb_epochs=20):
    """Training loop"""

    min_f1 = -torch.inf
    train_losses = []
    val_losses = []

    # Iterrate over epochs
    for e in range(nb_epochs):

        # Training
        train_loss = 0.0

        for i, batch in enumerate(tqdm(train_dataloader)):            
            
            #print(i)
            # unpack batch             
            labels = batch['label'].to(device)
            spans = batch['spans'].to(device)
            input_ids = batch['input_ids'].to(device)
            
            inputs = input_ids, spans
            
            # Reset gradients to 0
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(inputs)
            
            # Compute training loss
            current_loss = loss(outputs.flatten(0,1), labels.flatten())
            train_loss += current_loss.detach().item()

            # Compute gradients
            current_loss.backward()

            # Update weights
            optimizer.step()            
            
            del batch
        
        scheduler.step()
            
        
        # Validation
        val_loss = 0.0
        
        # torch.cuda.empty_cache()
        
        # Put model in eval mode
        model.eval()
        
        preds_l = []
        labels_l = []
        
        for batch in tqdm(val_dataloader):            
            
            # unpack batch             
            labels = batch['label'].to(device)
            spans = batch['spans'].to(device)
            input_ids = batch['input_ids'].to(device)
            
            inputs = input_ids, spans
            
            # Forward Pass
            outputs = model(inputs)

            # Compute validation loss
            current_loss = loss(outputs.flatten(0,1), labels.flatten())
            val_loss += current_loss.detach().item()
            
            preds_for_f1 = torch.argmax(outputs, dim=2).flatten().tolist()
            labels_for_f1 = labels.flatten().tolist()
            
            preds_l.append(preds_for_f1)
            labels_l.append(labels_for_f1)
            
            del batch
        
        # Prints
        
        preds_l = flatten_list(preds_l)
        labels_l = flatten_list(labels_l)
        
        preds_l, labels_l = remove_dummy_labels(preds_l, labels_l)
        
        f1_score_epoch = f1_score(preds_l, labels_l, average='macro')        
        
        print(f"Epoch {e+1}/{nb_epochs} \
                \t Training Loss: {train_loss/len(train_dataloader):.3f} \
                \t Validation Loss: {val_loss/len(val_dataloader):.3f} \
                \t F1 score: {f1_score_epoch}")
        
        train_losses.append(train_loss/len(train_dataloader))
        val_losses.append(val_loss/len(val_dataloader))
        

        # Save model if val loss decreases
        if f1_score_epoch > min_f1:

            min_f1 = f1_score_epoch
            torch.save(model.first_model.state_dict(), 'first_model.pt')
            torch.save(model.model_am.state_dict(), 'model_am.pt')
            torch.save(model.model_ac.state_dict(), 'model_ac.pt')
            torch.save(model.model_fts.state_dict(), 'model_fts.pt')
            torch.save(model.state_dict(), 'best_model.pt')
            
    return train_losses, val_losses

In [54]:
train_losses, val_losses = train(custom_model, loss, optimizer, train_dataloader, val_dataloader, NB_EPOCHS)

  7%|▋         | 1/14 [00:00<00:01,  6.52it/s]


RuntimeError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 15.74 GiB total capacity; 13.80 GiB already allocated; 37.56 MiB free; 14.64 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### Predictions

In [ ]:
# # Load best model
# network_2 = Network()

# network_2.load_state_dict(torch.load(path))
# network_2.eval()

In [ ]:
test_dataloader = DataLoader(dataset['test'], batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# first_model =  BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# first_model.load_state_dict(torch.load('first_model.pt'))

In [ ]:
first_model = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
first_model.load_state_dict(torch.load('first_model.pt'))

In [ ]:
model_am = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_am.load_state_dict(torch.load('model_am.pt'))

In [ ]:
model_ac = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_ac.load_state_dict(torch.load('model_ac.pt'))

In [ ]:
model_fts = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_fts.load_state_dict(torch.load('model_fts.pt'))

In [ ]:
# Load best model

# custom_model_2 = CustomBERTKuri(first_model, model_am, model_ac, model_fts, 3)
custom_model_2 = CustomBERTKuri(first_model, model_am, model_ac, model_fts, 2)
custom_model_2.load_state_dict(torch.load('best_model.pt'))

custom_model_2.to(device).eval()

In [ ]:
def predict(model, test_dataloader=None):
    
    """Prediction loop"""

    preds_l = []
    labels_l = []
    
    model.eval()

    for batch in test_dataloader:            
            
        # unpack batch             
        labels = batch['label'].to(device).flatten().tolist()
        spans = batch['spans'].to(device)
        input_ids = batch['input_ids'].to(device)
        
        inputs = input_ids, spans

        # get output
        
        raw_preds = model(inputs).to('cpu')
        # print(raw_preds.shape)
        raw_preds = raw_preds.detach()#.numpy()

        # Compute argmax
        
        predictions = torch.argmax(raw_preds, dim=2).flatten().tolist()
        preds_l.append(predictions)
        labels_l.append(labels)        
        
        del batch
            
    return flatten_list(preds_l), flatten_list(labels_l)

In [ ]:
#test_preds, test_labels = predict(custom_model, test_dataloader)

In [ ]:
#print(classification_report(test_labels, test_preds, digits=3))

In [ ]:
# remove -100s

In [ ]:
#test_preds_l, test_labels_l = remove_dummy_labels(test_preds, test_labels)

In [ ]:
#print(classification_report(test_labels_l, test_preds_l, digits=3))

In [ ]:
### -100 done!

In [ ]:
test_preds, test_labels = predict(custom_model_2, test_dataloader)

In [ ]:
print(classification_report(test_labels, test_preds, digits=3))

In [ ]:
test_preds_l, test_labels_l = remove_dummy_labels(test_preds, test_labels)

In [ ]:
print(classification_report(test_labels_l, test_preds_l, digits=3))

precision    recall  f1-score   support

           0      0.893     0.865     0.879       155
           1      0.663     0.683     0.673       303
           2      0.896     0.892     0.894       805

    accuracy                          0.838      1263
   macro avg      0.818     0.813     0.815      1263
weighted avg      0.840     0.838     0.839      1263

12 batch size, 0.006579332246575687